# Tuson Python Meetup Spark Demo
## Hello and welcome!

In this demo we will use this Jupyter Notebook to write a small program to parse an apache access log and then query the results.  This workshop is a hand-on class from teh previous demo, using the same data.  Spark is a large project with a huge surface area.  This demo will only focus on the SQL-like capabilities of Spark and offer code samples.

## You will need...
- Docker CE
  - Windows and Mac folks should download and install Docker CE using the provider installers from [Docker](https://store.docker.com/search?type=edition&offering=community)
  - Linux folks should install Docker using their distribution provided packages
- Git
  - Windows folks can download git [here](https://git-scm.com/downloads)
  - Mac folks should should have CLI git clients aleady 
  - Linux folks may need need install a seperate package
 
## Please raise your hand...
- if you have a question.
- if you are having difficulty.


In [ ]:
# Space... The Final Frontier...
#
# ... 
#
# These are the voyages of the the startship "don't read ahead" ....























# Step One
We need to import Spark and create a session


In [ ]:
import pyspark
from pyspark.sql import Row
from pyspark.sql import SparkSession

# This creates a spark context to operate in: use the local machine and all avalaible processors
sc = pyspark.SparkContext('local[*]')

# This create a session in which we can build tables and
spark = SparkSession(sc)

# Step Two
We need to parse a log data line which looks like this
```
109.169.248.247 - - [12/Dec/2015:18:25:11 +0100] "GET /administrator/ HTTP/1.1" 200 4263 "-" "Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20100101 Firefox/34.0" "-"
```
Into something meaningful like this:

| key | ip addr | identd | http auth | Datetime | Request (verb path proto) | Response | Bytes Sent | Referer | User-Agent | ? |
|-----|---------|---|---|----------|-----------------------------|---------------|------------|---|------------|---|
|line | `109.169.248.247` | `-` | `-` | `[12/Dec/2015:18:25:11 +0100]` | `"GET /administrator/ HTTP/1.1"` | `200` | `4263` | `"-"` | `"Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20100101 Firefox/34.0"` | `"-"` |
|regex|`^(\S+)` | `(\S+)` | `(\S+)` | `\[([\w:/]+\s[+\-]\d{4})\]` | `"(\S+) (\S+) (\S+)"` | `(\d{3})` | `(\d+)?` | `"(.*)"` | `"(.*)"` | `"-"` |

You will note that we don't know what that last field is.  It's often vhost, but in our dataset it's only `"-"` so we will assume it is some site specific field that is scrubbed and not of our concern.

In [ ]:
import re

APACHE_ACCESS_LOG_PATTERN = '^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+) (\S+)" (\d{3}) (\d+|-) "(.*)" "(.*)" "-"'


# Returns a dictionary containing the parts of the Apache Access Log.
def parse_apache_log_line(logline):
    match = re.search(APACHE_ACCESS_LOG_PATTERN, logline)
    if match is None:
        # Optionally, you can change this to just ignore if each line of data is not critical.
        # For this example, we want to ensure that the format is consistent.
        raise Exception("Invalid logline: %s" % logline)
    # This is a quirk of apache, if there is no value, it will often log a "-" instead to 
    # indicate a null value.  For size, this causes typing problems for our field, so 
    # fixup this field with a little processing logic
    if match.group(9) in (None, "-"):
        size = 0
    else:
        size = match.group(9)
    return Row(
        ipAddress    = str(match.group(1)),
        clientIdentd = str(match.group(2)),
        userId       = str(match.group(3)),
        dateTime     = str(match.group(4)),
        verb         = str(match.group(5)),
        path         = str(match.group(6)),
        protocol     = str(match.group(7)),
        responseCode = int(match.group(8)),
        contentSize  = int(size),
        referer      = str(match.group(10)),
        user_agent   = str(match.group(11))
    )

# Step Three
We need to load our log files

In [ ]:
log_files = "file:///home/jovyan/work/access.log"
raw_log_files = sc.textFile(log_files)

## Step Three and a Half
Once loaded, we need to parse them and make the result a SQL table for us to query

In [ ]:
parsed_log_files = raw_log_files.map(parse_apache_log_line)
parsed_log_files.toDF().registerTempTable("log_data")

# Step Four
We need to query our data

In [ ]:
sqlDF = spark.sql("SELECT count(*) AS count, ipAddress FROM log_data GROUP BY ipAddress ORDER BY count DESC")
sqlDF.show()

## What if we wanted to order by date?
Spark has a User Defined Function (UDF) for that.  UDF's are in java [SimpleDateFormat](https://docs.oracle.com/javase/6/docs/api/java/text/SimpleDateFormat.html)

In [ ]:
from pyspark.sql.functions import date_format

sqlDF = spark.sql("SELECT ipAddress, dateTime, to_timestamp(dateTime, 'dd/MMM/yyyy:HH:mm:ss Z') AS date FROM log_data ORDER BY date DESC LIMIT 10")
sqlDF.show()

## What if we wanted to run our data though an external program?
Let's assume we needed to process each log line in the RDD  with a custom program and then work on the resulting set.  Spark offers the pipe() transformation to process RDD's though an external program.  Pipe() expects the program to act as a FIFO with newline terminated lines going into stdin and new data emitted on stdout.

In [ ]:
log_md5_sums = raw_log_files.pipe("/usr/bin/rev")
log_md5_sums.collect()
# my external
print(log_md5_sums.take(1))
# mostly the same but with python (casting the result as sting stringifies the [] hence the weirdness at ends)
print(str(raw_log_files.take(1))[::-1])